In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print('o')
import tensorflow as tf

print(tf.__version__)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

o
2.1.0


In [2]:
ls "../input/usos-del-suelo-desde-el-espacio/modelos/"

modeloXception-96-v2.h5


In [6]:
from tensorflow.keras.models import load_model

model = load_model("../input/usos-del-suelo-desde-el-espacio/modelos/modeloXception-96-v2.h5")

"""
----------Cómo hemos generado el modelo guardado:--------------------------

from tensorflow.keras.applications.xception import Xception

xception = Xception(
    input_shape=(256,256,3),
    include_top=False,
    weights="imagenet"
)

for layer in inception_v3.layers:
    layer.trainable=False
    
xception.summary()

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D, Activation, MaxPooling2D

model = tf.keras.models.Sequential([
    xception,
    
    Flatten(),
    Dense(512,activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128,activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    Dense(45, activation='softmax')
])

"""

'\n----------Cómo hemos generado el modelo guardado:--------------------------\n\nfrom tensorflow.keras.applications.xception import Xception\n\nxception = Xception(\n    input_shape=(256,256,3),\n    include_top=False,\n    weights="imagenet"\n)\n\nfor layer in inception_v3.layers:\n    layer.trainable=False\n    \nxception.summary()\n\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D, Activation, MaxPooling2D\n\nmodel = tf.keras.models.Sequential([\n    xception,\n    \n    Flatten(),\n    Dense(512,activation="relu"),\n    BatchNormalization(),\n    Dropout(0.5),\n    \n    Dense(128,activation="relu"),\n    BatchNormalization(),\n    Dropout(0.5),\n    Dense(45, activation=\'softmax\')\n])\n\n'

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 8, 8, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               67109376  
_________________________________________________________________
batch_normalization_10 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
batch_normalization_11 (Batc (None, 128)              

In [ ]:
# Base learning rate
base_learning_rate = 0.0001
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [7]:
path = "../input/usos-del-suelo-desde-el-espacio/"
train_path = path + "train/train/"
test_path = path + "validacion/"

In [8]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   zoom_range=0.5,
                                   rotation_range=45,
                                  validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1/255)
batch_size = 64

train_generator= train_datagen.flow_from_directory(
    train_path,
    target_size=(256, 256),
    batch_size=batch_size,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(256,256),
    batch_size=batch_size,
    subset='validation') # set as validation data

test_generator= test_datagen.flow_from_directory(
    test_path,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 19727 images belonging to 45 classes.
Found 4930 images belonging to 45 classes.
Found 3080 images belonging to 1 classes.


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

history = model.fit(
    train_generator,
    epochs=50,
    verbose=1,
    callbacks=[callback],
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data = validation_generator
)

Train for 308 steps, validate for 78 steps
Epoch 1/50
307/308 [============================>.] - ETA: 1s - loss: 2.8612 - accuracy: 0.9838

In [ ]:
model.save("../working/modeloXception.h5")

In [ ]:
preds = model.predict(test_generator, verbose = 1)


In [ ]:
import glob,os
import pandas as pd
import numpy as np

filename = "../working/" + "output.csv"
filename2 = "../working/" + "output_con_prob.csv"

ids = [f[11:] for f in test_generator.filenames]

#Guardas el fichero csv para kaggle
solution = pd.DataFrame(data = {"ID": ids, "Category": np.argmax(preds, axis = 1)})
solution.to_csv(filename, index = False, columns = ["ID", "Category"])

#Guardar la probabilidad de pertenencia a la clase predicha
solution2 = pd.DataFrame(data = {"ID": ids, "Category": np.argmax(preds, axis = 1), "Pred":  np.amax(preds, axis = 1)})
solution2.to_csv(filename2, index = False, columns = ["ID", "Category","Pred"])